# Hacer predicciones en base a la ruta de una imagen local
A01369630 Julieta Itzel Pichardo Meza

Pasos para usar este Notebook de forma local
1. Crear el entorno usando environment.yml
    - Abrir una terminal (por ejemplo, la terminal integrada en Visual Studio Code o Anaconda Prompt).
    - Ubicarse en la carpeta donde se encuentra el archivo environment.yml .
    - Crear el entorno ejecutando:
        ~~~
        conda env create -f environment.yml
        ~~~
    - Activar el entorno recién creado:
        ~~~
        conda activate perros-gatos-pytorch
        ~~~

Este proceso instalará automáticamente todas las dependencias necesarias (Python, PyTorch, Pillow, NumPy, Matplotlib, etc.) sin requerir instalaciones manuales.

2. Registrar el entorno como kernel de Jupyter
    - Con el entorno activado, registrar el kernel para que VSCode o Jupyter Notebook puedan detectarlo:
        ~~~
        python -m ipykernel install --user --name perros-gatos-pytorch --display-name "Python (perros-gatos-pytorch)"
        ~~~
3. Abrir el Notebook y seleccionar el kernel
    - En VSCode: Kernel → Change Kernel → Python (perros-gatos-pytorch)

    - En Jupyter Notebook/JupyterLab: Seleccionar el kernel con el nombre Python (perros-gatos-pytorch).

4. Ejecutar las celdas del Notebook en orden
- Celda 1

    - Verificar que imprima: “Usando dispositivo: cpu”
    - Esta celda importa librerías, configura las transformaciones y fuerza el uso de CPU para evitar errores.

- Celda 2
    - Verificar que imprima: “Modelo cargado correctamente desde: clasificador_perros_gatos_v3.pth”
    - Esta celda no debe fallar a menos que el archivo .pth no esté en el mismo directorio o la ruta sea incorrecta.

- Celda 3
    - Define la función predict_image_no_plot.
    - No imprime nada a menos que ocurra un error.

- Celda 4
    - Define la función predict_from_path_string.
    - Tampoco imprime nada.

- Celda 5
    - Editar esta línea para indicar la ruta de la imagen que se desea predecir:
        ~~~
        ruta_imagen = r"C:\Users\julyp\Downloads\tests\nueve.jpg"
        ~~~

    - Puedes usar:
        - Ruta relativa, si la imagen está en la misma carpeta que el Notebook.
        - Ruta absoluta, si está en otro directorio.
        - Al ejecutarla, esta celda imprimirá un resultado como:
            >>> Predicción: dog (probabilidad: 0.932)

In [ ]:
# Celda 1:
# * Importa librerías necesarias para inferencia.
# * Fuerza el uso de CPU (desactiva CUDA).
# * Define las clases del modelo (cat/dog).
# * Configura las transformaciones de preprocesamiento que se aplican a cada imagen.

import os

# Desactivar CUDA explícitamente por si está mal configurado y tira el kernel
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np

# Fuerza a CPU
device = torch.device("cpu")
print("Usando dispositivo:", device)

# Clases (ajusta el orden si es necesario)
class_names = ["cat", "dog"]  # o ["dog", "cat"]
num_classes = len(class_names)

# Transformaciones de inferencia: las mismas que en entrenamiento de v3
inference_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # o el tamaño que usaste
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet
        std=[0.229, 0.224, 0.225]
    ),
])


Usando dispositivo: cpu


In [ ]:
# Celda 2:
# - Crea el modelo v3 basado en ResNet preentrenada.
# - Reemplaza la capa final para que tenga 2 salidas (cat/dog).
# - Carga los pesos entrenados desde el archivo .pth.
# - Envía el modelo a CPU y lo pone en modo evaluación.

def create_model_v3(num_classes=2):
    """
    Crea el modelo de la versión 3 usando ResNet preentrenada
    y reemplaza la capa fully-connected final.
    """
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    return model

# Ruta donde guardaste los pesos de la versión 3
weights_path = "clasificador_perros_gatos_v3.pth"  # cambia si se llama diferente

model_v3 = create_model_v3(num_classes=num_classes)

# Cargar pesos siempre en CPU
state_dict = torch.load(weights_path, map_location="cpu")
model_v3.load_state_dict(state_dict)

model_v3.to(device)
model_v3.eval()

print("Modelo v3 cargado correctamente desde:", weights_path)


Modelo v3 cargado correctamente desde: clasificador_perros_gatos_v3.pth


In [ ]:
# Celda 3:
# - Define la función predict_image_no_plot.
# - Carga una imagen desde la ruta indicada.
# - Aplica las transformaciones de inferencia.
# - Ejecuta el modelo en modo evaluación sin gradientes.
# - Calcula la probabilidad y la clase predicha.
# - Devuelve la etiqueta y la probabilidad, sin mostrar la imagen.

def predict_image_no_plot(image_path, model, transform, class_names, device):
    """
    Realiza una predicción sobre una imagen individual.
    No dibuja nada, solo devuelve (pred_label, pred_prob).
    """
    from pathlib import Path

    image_path = Path(image_path)
    if not image_path.is_file():
        raise FileNotFoundError(f"No se encontró la imagen: {image_path}")

    # Cargar imagen
    img = Image.open(image_path).convert("RGB")

    # Transformaciones
    img_tensor = transform(img)
    img_tensor = img_tensor.unsqueeze(0)  # (1, C, H, W)
    img_tensor = img_tensor.to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(img_tensor)
        probs = torch.softmax(outputs, dim=1)
        prob, pred_idx = torch.max(probs, dim=1)

    pred_label = class_names[pred_idx.item()]
    pred_prob = prob.item()

    return pred_label, pred_prob


In [ ]:
# Celda 4:
# - Define la función predict_from_path_string.
# - Recibe una ruta de imagen en formato string.
# - Llama a predict_image_no_plot usando el modelo v3 y las transformaciones.
# - Imprime en pantalla la clase predicha y su probabilidad.
# - Maneja errores y muestra mensajes si algo falla (por ejemplo, ruta inválida).

def predict_from_path_string(path_str):
    try:
        pred_label, pred_prob = predict_image_no_plot(
            image_path=path_str,
            model=model_v3,
            transform=inference_transform,
            class_names=class_names,
            device=device
        )
        print(f">>> Predicción: {pred_label} (probabilidad: {pred_prob:.3f})")
    except Exception as e:
        print(f"Error al procesar la imagen: {e}")


In [ ]:
# Celda 5:
# - Especifica la ruta de la imagen que se quiere clasificar.
# - Llama a predict_from_path_string para obtener la predicción.
# - Esta es la celda que se modifica para probar nuevas imágenes.

ruta_imagen = r"C:\Users\julyp\Downloads\tests\nueve.jpg"

predict_from_path_string(ruta_imagen)


>>> Predicción: dog (probabilidad: 0.846)
